In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
import pandas as pd
import numpy as np
import pickle
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.linear_model import Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
import lightgbm as lgb
import joblib
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages

In [2]:
# 读取数据
df = pd.read_csv(r"D:\回归数据\allpi3k分子描述符内部数据集.csv",encoding = 'gbk')

In [3]:
# 设定递归特征选择后的分子描述符
selected_features = ['BertzCT', 'Chi2v', 'Chi4n', 'Chi4v', 
                     'SMR_VSA3', 'SMR_VSA9', 'SlogP_VSA8', 
                     'TPSA', 'MolMR', 'fr_NH0']


In [4]:
# 计算摩根指纹
fingerprints = []
for s in df['smiles']:
    mol = Chem.MolFromSmiles(s)
    if mol is None:
        fingerprints.append(None)
    else:
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048)
        arr = np.zeros((2048,), dtype=int)
        DataStructs.ConvertToNumpyArray(fp, arr)
        fingerprints.append(arr)

df['fingerprint'] = fingerprints
df = df[df['fingerprint'].notnull()].reset_index(drop=True)  # 删除无效 SMILES
fingerprint_array = np.array(df['fingerprint'].tolist())  # 转换为 NumPy 数组

# 提取分子描述符
descriptors = df[selected_features]


[14:43:26] DEPRECATION WARNING: please use MorganGenerator
[14:43:26] DEPRECATION WARNING: please use MorganGenerator
[14:43:26] DEPRECATION WARNING: please use MorganGenerator
[14:43:26] DEPRECATION WARNING: please use MorganGenerator
[14:43:26] DEPRECATION WARNING: please use MorganGenerator
[14:43:26] DEPRECATION WARNING: please use MorganGenerator
[14:43:26] DEPRECATION WARNING: please use MorganGenerator
[14:43:26] DEPRECATION WARNING: please use MorganGenerator
[14:43:26] DEPRECATION WARNING: please use MorganGenerator
[14:43:26] DEPRECATION WARNING: please use MorganGenerator
[14:43:26] DEPRECATION WARNING: please use MorganGenerator
[14:43:26] DEPRECATION WARNING: please use MorganGenerator
[14:43:26] DEPRECATION WARNING: please use MorganGenerator
[14:43:26] DEPRECATION WARNING: please use MorganGenerator
[14:43:26] DEPRECATION WARNING: please use MorganGenerator
[14:43:26] DEPRECATION WARNING: please use MorganGenerator
[14:43:26] DEPRECATION WARNING: please use MorganGenerat

In [5]:
from rdkit import Chem
from rdkit.Chem import MACCSkeys
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

# 计算 MACCS 指纹
def compute_maccs(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        maccs = MACCSkeys.GenMACCSKeys(mol)
        return np.array(maccs)
    else:
        return np.zeros((167,))  # MACCS指纹长度为167

# 计算MACCS指纹数组
maccs_fingerprints = np.array([compute_maccs(smiles) for smiles in df['smiles']])

# 标准化分子描述符
scaler = StandardScaler()
X_scaled = scaler.fit_transform(descriptors)

# 合并摩根指纹、MACCS指纹和标准化的描述符
X_combined = np.hstack([X_scaled, fingerprint_array, maccs_fingerprints])
y = df['pIC50'].values

X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)


In [6]:
import joblib
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
import lightgbm as lgb
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import pandas as pd

# 定义回归模型
models = {
    "ETR": ExtraTreesRegressor(n_estimators=30, max_depth=15, min_samples_split=2, min_samples_leaf=2),
    "RFR": RandomForestRegressor(n_estimators=100, random_state=42),
    "GBR": GradientBoostingRegressor(n_estimators=15, learning_rate=0.5, max_depth=7),
    "SVR": SVR(kernel='rbf', C=1, gamma=1, epsilon=0.1),
    "MLP": MLPRegressor(hidden_layer_sizes=(150, 10), activation='relu', solver='adam', max_iter=200, random_state=42),
    "KNN": KNeighborsRegressor(n_neighbors=3, p=2, weights='uniform', leaf_size=2),
    "XGB": XGBRegressor(n_estimators=200, learning_rate=0.05),
    "LightGBM": None
}

train_results = []
test_results = []

# 训练并评估各个模型，并保存模型
for model_name, model in models.items():
    if model is not None:
        model.fit(X_train, y_train)
        
        # 保存模型
        joblib.dump(model, f'{model_name}.pkl')
        
        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)
        train_r2 = round(r2_score(y_train, y_pred_train), 4)
        train_mse = round(mean_squared_error(y_train, y_pred_train), 4)
        train_mae = round(mean_absolute_error(y_train, y_pred_train), 4)
        test_r2 = round(r2_score(y_test, y_pred_test), 4)
        test_mse = round(mean_squared_error(y_test, y_pred_test), 4)
        test_mae = round(mean_absolute_error(y_test, y_pred_test), 4)
        
        train_results.append([model_name, train_r2, train_mse, train_mae])
        test_results.append([model_name, test_r2, test_mse, test_mae])
    else:
        # LightGBM 特殊处理
        lgb_train = lgb.Dataset(X_train, label=y_train)
        lgb_test = lgb.Dataset(X_test, label=y_test)
        params = {
            'objective':'regression',
            'num_leaves': 30,
            'learning_rate': 0.1,
            'feature_fraction': 0.6,
            'bagging_fraction': 0.9,
            'bagging_freq': 7
        }
        callbacks = [lgb.early_stopping(stopping_rounds=10)]
        lgb_model = lgb.train(params, lgb_train, valid_sets=[lgb_test], num_boost_round=100, callbacks=callbacks)
        
        # 保存 LightGBM 模型
        lgb_model.save_model('LightGBM_model.txt')
        
        y_pred_train = lgb_model.predict(X_train)
        y_pred_test = lgb_model.predict(X_test)
        train_r2 = round(r2_score(y_train, y_pred_train), 4)
        train_mse = round(mean_squared_error(y_train, y_pred_train), 4)
        train_mae = round(mean_absolute_error(y_train, y_pred_train), 4)
        test_r2 = round(r2_score(y_test, y_pred_test), 4)
        test_mse = round(mean_squared_error(y_test, y_pred_test), 4)
        test_mae = round(mean_absolute_error(y_test, y_pred_test), 4)
        
        train_results.append([model_name, train_r2, train_mse, train_mae])
        test_results.append([model_name, test_r2, test_mse, test_mae])

# 打印训练和测试结果
train_df = pd.DataFrame(train_results, columns=["Model", "R2", "MSE", "MAE"])
test_df = pd.DataFrame(test_results, columns=["Model", "R2", "MSE", "MAE"])

print("Training Set Results:")
print(train_df)
print("Testing Set Results:")
print(test_df)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.021915 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5938
[LightGBM] [Info] Number of data points in the train set: 8324, number of used features: 1881
[LightGBM] [Info] Start training from score 6.972400
Training until validation scores don't improve for 10 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.441897
Training Set Results:
      Model      R2     MSE     MAE
0       ETR  0.7513  0.3756  0.4427
1       RFR  0.9574  0.0643  0.1789
2       GBR  0.7744  0.3408  0.4431
3       SVR  0.8193  0.2729  0.3444
4       MLP  0.9871  0.0195  0.0854
5       KNN  0.8557  0.2179  0.3259
6       XGB  0.7736  0.3419  0.4482
7  LightGBM  0.7968  0.3069  0.4189
Testing Set Results:
      Model      R2     MSE     MAE
0       ETR  0.6066  0.5933  0.5789
1       RFR  0.7

In [9]:
import os
import numpy as np
import pandas as pd
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, MACCSkeys
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import joblib
import lightgbm as lgb

# 1️⃣ 加载外部验证集
external_csv = r"D:\回归数据\allpi3k分子描述符外部验证集.csv"
df_ext = pd.read_csv(external_csv, encoding="gbk")

# 2️⃣ 计算摩根指纹和MACCS指纹并整合
fingerprints_ext = []
maccs_ext = []

for s in df_ext['smiles']:
    mol = Chem.MolFromSmiles(s)
    if mol is None:
        fingerprints_ext.append(None)
        maccs_ext.append(None)
    else:
        # 摩根指纹
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048)
        arr = np.zeros((2048,), dtype=int)
        DataStructs.ConvertToNumpyArray(fp, arr)
        fingerprints_ext.append(arr)

        # MACCS指纹
        maccs_fp = MACCSkeys.GenMACCSKeys(mol)
        maccs_arr = np.zeros((167,), dtype=int)
        DataStructs.ConvertToNumpyArray(maccs_fp, maccs_arr)
        maccs_ext.append(maccs_arr)

df_ext['fingerprint'] = fingerprints_ext
df_ext['maccs'] = maccs_ext

# 删除指纹计算失败的分子
df_ext = df_ext[df_ext['fingerprint'].notnull() & df_ext['maccs'].notnull()].reset_index(drop=True)

# 3️⃣ 将指纹转换为数组
fingerprint_array_ext = np.array(df_ext['fingerprint'].tolist())
maccs_array_ext = np.array(df_ext['maccs'].tolist())

# 4️⃣ 特征提取：分子描述符 + 摩根指纹 + MACCS指纹
descriptor_cols = ['BertzCT', 'Chi2v', 'Chi4n', 'Chi4v', 'SMR_VSA3', 'SMR_VSA9', 'SlogP_VSA8', 'TPSA', 'MolMR', 'fr_NH0']
descriptors_ext = df_ext[descriptor_cols]

# 5️⃣ 标准化
scaler = StandardScaler()
descriptors_ext_scaled = scaler.fit_transform(descriptors_ext)

# 6️⃣ 整合特征
X_combined_ext = np.hstack([descriptors_ext_scaled, fingerprint_array_ext, maccs_array_ext])

# 7️⃣ 提取 pIC50 值
y_external = df_ext["pIC50"]

# 8️⃣ 加载模型的函数
def load_model(model_filename):
    if not os.path.exists(model_filename):
        raise FileNotFoundError(f"Model file '{model_filename}' does not exist.")
    return lgb.Booster(model_file=model_filename) if model_filename.endswith(".txt") else joblib.load(model_filename)

# 9️⃣ 评估模型的函数
def evaluate_model(model, X_combined_ext, y_external):
    y_pred = model.predict(X_combined_ext)
    mse = mean_squared_error(y_external, y_pred)
    r2 = r2_score(y_external, y_pred)
    mae = mean_absolute_error(y_external, y_pred)
    return mse, r2, mae

# 🔟 需要评估的模型列表
model_filenames = [
    'ETR.pkl', 'RFR.pkl', 'GBR.pkl', 'SVR.pkl', 
    'MLP.pkl', 'KNN.pkl', 'XGB.pkl', 'LightGBM_model.txt'
]

# 1️⃣1️⃣ 评估所有模型
performance_comparison = []
for filename in model_filenames:
    try:
        model = load_model(filename)
        mse_external, r2_external, mae_external = evaluate_model(model, X_combined_ext, y_external)
        performance_comparison.append({
            'Model': filename,
            'MSE': mse_external,
            'R²': r2_external,
            'MAE': mae_external
        })
    except Exception as e:
        print(f"Error with model {filename}: {e}")

# 1️⃣2️⃣ 打印结果
print("External Validation Results:")
print(pd.DataFrame(performance_comparison))


[13:57:01] DEPRECATION WARNING: please use MorganGenerator
[13:57:01] DEPRECATION WARNING: please use MorganGenerator
[13:57:01] DEPRECATION WARNING: please use MorganGenerator
[13:57:01] DEPRECATION WARNING: please use MorganGenerator
[13:57:01] DEPRECATION WARNING: please use MorganGenerator
[13:57:01] DEPRECATION WARNING: please use MorganGenerator
[13:57:01] DEPRECATION WARNING: please use MorganGenerator
[13:57:01] DEPRECATION WARNING: please use MorganGenerator
[13:57:01] DEPRECATION WARNING: please use MorganGenerator
[13:57:01] DEPRECATION WARNING: please use MorganGenerator
[13:57:01] DEPRECATION WARNING: please use MorganGenerator
[13:57:01] DEPRECATION WARNING: please use MorganGenerator
[13:57:01] DEPRECATION WARNING: please use MorganGenerator
[13:57:01] DEPRECATION WARNING: please use MorganGenerator
[13:57:01] DEPRECATION WARNING: please use MorganGenerator
[13:57:01] DEPRECATION WARNING: please use MorganGenerator
[13:57:01] DEPRECATION WARNING: please use MorganGenerat

External Validation Results:
                Model       MSE        R²       MAE
0             ETR.pkl  0.610697  0.608187  0.576638
1             RFR.pkl  0.485880  0.688267  0.502557
2             GBR.pkl  0.592788  0.619677  0.579337
3             SVR.pkl  1.504730  0.034590  1.016825
4             MLP.pkl  0.644596  0.586438  0.578284
5             KNN.pkl  0.506441  0.675076  0.495092
6             XGB.pkl  0.569219  0.634798  0.570635
7  LightGBM_model.txt  0.520963  0.665758  0.539813
